# xarray

还有一个数据处理的库非常好用 -- xarray。本文参考：[xarray官方文档](http://xarray.pydata.org/en/stable/)，简单记录下xarray的基本使用方法。

xarray是一个专门用来处理多维标签数据的python库。Xarray在类似于**NumPy的原始数组上以维度，坐标和属性的形式引入标签**，从而提供了更直观，更简洁和更少出错的开发人员体验。引入标签的原因是现实世界中，数据往往不是原始的多维数组，而还包含一系列反映数据编码信息的标签，比如时空信息。xarray可以利用这些标签更好地完成对数据的操作，后面会根据实际使用逐步记录。

Xarray借鉴了pandas，特别适合处理netCDF文件，netcdf是xarray数据模型的来源，如果不了解netcdf可以参考：[netcdf-python 介绍](https://github.com/OuyangWenyu/aqualord/blob/master/DataFormat/netcdf.ipynb)。xarray中最基本的数据结构是DataArray和Dataset，前者类似于pandas里的Series，后者类似DataFrame，更多信息后面记录。

另外xarray还与dask紧密集成以进行并行计算，后面用到dask再做记录。

conda安装（本repo environment.yml文件已包括）：

```Shell
conda install -c conda-forge xarray
```

## 基本术语

以下无明确说明时，arr 表示DataArray对象。

- DataArray：是一个多维数组，有标签，如果name属性设置了，就成为一个named DataArray
- Dataset：一个类dict的DataArray对象集合，有多个排列的维度。Datasets有 Varaible
- Varaible：类似于netcdf的variable，包含维度，数据，属性等。variables和numpy数组的主要区别是变量上的广播运算基于维度名称的。每个DataArray有一个潜在variable可以通过arr.variable访问。Variable在Dataset和DataArray内的，所以不必单独用它。
- Dimension：一个维度轴就是固定一个维度上的所有点的集合。每个维度轴有一个名字，比如x维度。DataArray对象的维度就是被命名的维度轴。第i个维度名称可用arr.dim[i]获取。默认的维度名称是dim_0,dim_1以此类推。
- Coordinate：一个标记维度的数组。在一维情况下，坐标数组的至可以被认为是维度的标签。有两种坐标：
    - Dimension coordinate：一维的坐标数组用名字和维度名字指定给arr，可见于arr.dims。维度坐标类似于DataFrame中的index。
    - Non-dimension coordinate：非维度坐标可见于arr.coords，这些坐标数组可以是一维也可以是多维，多维情况主要见于物理坐标和逻辑坐标不一致的时候，非维度坐标是不能索引的。
- Index：index是优化数据结构用于快速索引和切片的。xarray用维度坐标可以i快速索引。

以上有些晦涩，接下来看例子。

## 基本数据结构

先看DataArray，多维标记数组。

### DataArray

In [1]:
import numpy as np
data = np.random.rand(4, 3)
data

array([[0.99800627, 0.10697747, 0.53674625],
       [0.2134859 , 0.17025178, 0.42545607],
       [0.34115689, 0.05752575, 0.86193741],
       [0.73160757, 0.91303752, 0.2315763 ]])

In [2]:
import pandas as pd
locs = ['IA', 'IL', 'IN']
times = pd.date_range('2000-01-01', periods=4)
times

DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03', '2000-01-04'], dtype='datetime64[ns]', freq='D')

In [3]:
import xarray as xr
foo = xr.DataArray(data, coords=[times, locs], dims=['time', 'space'])
foo

<xarray.DataArray (time: 4, space: 3)>
array([[0.99800627, 0.10697747, 0.53674625],
       [0.2134859 , 0.17025178, 0.42545607],
       [0.34115689, 0.05752575, 0.86193741],
       [0.73160757, 0.91303752, 0.2315763 ]])
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03 2000-01-04
  * space    (space) <U2 'IA' 'IL' 'IN'

可以看到这个arr有两个坐标，一个时间，一个空间，时间对应一个坐标数组，是pandas的data_range，空间对应一个list，相应的数据有4 * 3=12个，且数组共4行，对应4个时间，三列对应三个空间。

最简单的初始化方式是直接使用data'：

In [4]:
xr.DataArray(data)

<xarray.DataArray (dim_0: 4, dim_1: 3)>
array([[0.99800627, 0.10697747, 0.53674625],
       [0.2134859 , 0.17025178, 0.42545607],
       [0.34115689, 0.05752575, 0.86193741],
       [0.73160757, 0.91303752, 0.2315763 ]])
Dimensions without coordinates: dim_0, dim_1

可以看到默认的坐标。

另外，可以直接使用字典形式创建coords：

In [5]:
xr.DataArray(data, coords=[('time', times), ('space', locs)])

<xarray.DataArray (time: 4, space: 3)>
array([[0.99800627, 0.10697747, 0.53674625],
       [0.2134859 , 0.17025178, 0.42545607],
       [0.34115689, 0.05752575, 0.86193741],
       [0.73160757, 0.91303752, 0.2315763 ]])
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03 2000-01-04
  * space    (space) <U2 'IA' 'IL' 'IN'

再比如多维的：

In [6]:
xr.DataArray(data, coords={'time': times, 'space': locs, 'const': 42, 
                           'ranking': (('time', 'space'), np.arange(12).reshape(4,3))},
             dims=['time', 'space'])

<xarray.DataArray (time: 4, space: 3)>
array([[0.99800627, 0.10697747, 0.53674625],
       [0.2134859 , 0.17025178, 0.42545607],
       [0.34115689, 0.05752575, 0.86193741],
       [0.73160757, 0.91303752, 0.2315763 ]])
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03 2000-01-04
  * space    (space) <U2 'IA' 'IL' 'IN'
    const    int32 42
    ranking  (time, space) int32 0 1 2 3 4 5 6 7 8 9 10 11

如上，time和space是维度坐标，const和ranking就是非维度坐标。维度是直接和数据对应上的，其他非维度的坐标是其他和数据不直接对应的。

另外，还可以使用DataFrame来初始化：

In [7]:
df = pd.DataFrame({'x': [0, 1], 'y': [2, 3]}, index=['a', 'b'])
df

,x,y
a,0,2
b,1,3


In [8]:
df.index.name = 'abc'
df

,x,y
abc,,
a,0,2
b,1,3


In [9]:
df.columns.name = 'xyz'
df

xyz,x,y
abc,,
a,0,2
b,1,3


In [10]:
xr.DataArray(df)

<xarray.DataArray (abc: 2, xyz: 2)>
array([[0, 2],
       [1, 3]], dtype=int64)
Coordinates:
  * abc      (abc) object 'a' 'b'
  * xyz      (xyz) object 'x' 'y'

接下来看看数组的属性：

In [11]:
foo.values

array([[0.99800627, 0.10697747, 0.53674625],
       [0.2134859 , 0.17025178, 0.42545607],
       [0.34115689, 0.05752575, 0.86193741],
       [0.73160757, 0.91303752, 0.2315763 ]])

注意DataArray中的数组值都是统一的数据类型。如果需要不同数据类型的，那么需要使用Dataset。

In [12]:
foo.dims

('time', 'space')

In [13]:
foo.coords

Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03 2000-01-04
  * space    (space) <U2 'IA' 'IL' 'IN'

In [14]:
foo.attrs

{}

In [15]:
print(foo.name)

None


以上有缺失默认值的，可以使用下列方式补充：

In [16]:
foo.name = 'foo'
foo.attrs['units'] = 'meters'
foo

<xarray.DataArray 'foo' (time: 4, space: 3)>
array([[0.99800627, 0.10697747, 0.53674625],
       [0.2134859 , 0.17025178, 0.42545607],
       [0.34115689, 0.05752575, 0.86193741],
       [0.73160757, 0.91303752, 0.2315763 ]])
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03 2000-01-04
  * space    (space) <U2 'IA' 'IL' 'IN'
Attributes:
    units:    meters

使用rename会返回一个新的数据数组：

In [17]:
foo.rename('bar')

<xarray.DataArray 'bar' (time: 4, space: 3)>
array([[0.99800627, 0.10697747, 0.53674625],
       [0.2134859 , 0.17025178, 0.42545607],
       [0.34115689, 0.05752575, 0.86193741],
       [0.73160757, 0.91303752, 0.2315763 ]])
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03 2000-01-04
  * space    (space) <U2 'IA' 'IL' 'IN'
Attributes:
    units:    meters

坐标是类似dict类型的

In [18]:
foo.coords['time']

<xarray.DataArray 'time' (time: 4)>
array(['2000-01-01T00:00:00.000000000', '2000-01-02T00:00:00.000000000',
       '2000-01-03T00:00:00.000000000', '2000-01-04T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03 2000-01-04

In [19]:
foo['time']

<xarray.DataArray 'time' (time: 4)>
array(['2000-01-01T00:00:00.000000000', '2000-01-02T00:00:00.000000000',
       '2000-01-03T00:00:00.000000000', '2000-01-04T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03 2000-01-04

坐标可以被删除：

In [20]:
foo['ranking'] = ('space', [1, 2, 3])
foo.coords

Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03 2000-01-04
  * space    (space) <U2 'IA' 'IL' 'IN'
    ranking  (space) int32 1 2 3

In [21]:
del foo['ranking']
foo.coords

Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03 2000-01-04
  * space    (space) <U2 'IA' 'IL' 'IN'

稍微小结下 DataArray：首先主体还是一个数组，数组的坐标直接对应维度坐标，比如一个二维的数组，那么就有两个维度坐标，数组中每个数据都唯一对应一对维度坐标值；还可以有非维度坐标，另外还有描述DataArray属性的。

下面看Dataset，它就是为netcdf定制的。可以简单地认为 Dataset 就是多个具有相同坐标系的 DataArray 的组合体。

### Dataset

In [22]:
temp = 15 + 8 * np.random.randn(2, 2, 3)
temp

array([[[23.18618225, 16.62877253, 20.2934517 ],
        [12.64546683,  7.9220311 , 16.94113908]],

       [[ 3.2112235 ,  5.29999304, 10.02171398],
        [13.48221583,  0.94690607, 18.03473387]]])

In [23]:
precip = 10 * np.random.rand(2, 2, 3)
precip 

array([[[6.16307358, 9.75041979, 3.83807332],
        [8.3555005 , 4.40372056, 1.91066065]],

       [[2.10114698, 6.13556892, 9.58585718],
        [1.98922117, 1.39673257, 4.39125251]]])

In [24]:
lon = [[-99.83, -99.32], [-99.79, -99.23]]
lat = [[42.25, 42.21], [42.63, 42.59]]
ds = xr.Dataset({'temperature': (['x', 'y', 'time'],  temp),
                     'precipitation': (['x', 'y', 'time'], precip)},
                    coords={'lon': (['x', 'y'], lon),
                            'lat': (['x', 'y'], lat),
                            'time': pd.date_range('2014-09-06', periods=3),
                            'reference_time': pd.Timestamp('2014-09-05')})
ds

<xarray.Dataset>
Dimensions:         (time: 3, x: 2, y: 2)
Coordinates:
    lon             (x, y) float64 -99.83 -99.32 -99.79 -99.23
    lat             (x, y) float64 42.25 42.21 42.63 42.59
  * time            (time) datetime64[ns] 2014-09-06 2014-09-07 2014-09-08
    reference_time  datetime64[ns] 2014-09-05
Dimensions without coordinates: x, y
Data variables:
    temperature     (x, y, time) float64 23.19 16.63 20.29 ... 0.9469 18.03
    precipitation   (x, y, time) float64 6.163 9.75 3.838 ... 1.989 1.397 4.391

可以直接传入DataArray到Dataset，或者DataFrame。

In [25]:
xr.Dataset({'bar': foo})

<xarray.Dataset>
Dimensions:  (space: 3, time: 4)
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03 2000-01-04
  * space    (space) <U2 'IA' 'IL' 'IN'
Data variables:
    bar      (time, space) float64 0.998 0.107 0.5367 ... 0.7316 0.913 0.2316

In [34]:
type(foo.to_pandas())

pandas.core.frame.DataFrame

In [26]:
xr.Dataset({'bar': foo.to_pandas()})

<xarray.Dataset>
Dimensions:  (space: 3, time: 4)
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03 2000-01-04
  * space    (space) object 'IA' 'IL' 'IN'
Data variables:
    bar      (time, space) float64 0.998 0.107 0.5367 ... 0.7316 0.913 0.2316

variable 就是 组成 Dataset 的 DataArray 的名字，也就是上面初始化时候 用的 dict 的 key。

像dict中使用key那样在Dataset中使用 variable 即可取出 DataArray：

In [27]:
'temperature' in ds

True

In [28]:
ds['temperature']

<xarray.DataArray 'temperature' (x: 2, y: 2, time: 3)>
array([[[23.18618225, 16.62877253, 20.2934517 ],
        [12.64546683,  7.9220311 , 16.94113908]],

       [[ 3.2112235 ,  5.29999304, 10.02171398],
        [13.48221583,  0.94690607, 18.03473387]]])
Coordinates:
    lon             (x, y) float64 -99.83 -99.32 -99.79 -99.23
    lat             (x, y) float64 42.25 42.21 42.63 42.59
  * time            (time) datetime64[ns] 2014-09-06 2014-09-07 2014-09-08
    reference_time  datetime64[ns] 2014-09-05
Dimensions without coordinates: x, y

或者：

In [33]:
ds.temperature

<xarray.DataArray 'temperature' (x: 2, y: 2, time: 3)>
array([[[23.18618225, 16.62877253, 20.2934517 ],
        [12.64546683,  7.9220311 , 16.94113908]],

       [[ 3.2112235 ,  5.29999304, 10.02171398],
        [13.48221583,  0.94690607, 18.03473387]]])
Coordinates:
    lon             (x, y) float64 -99.83 -99.32 -99.79 -99.23
    lat             (x, y) float64 42.25 42.21 42.63 42.59
  * time            (time) datetime64[ns] 2014-09-06 2014-09-07 2014-09-08
    reference_time  datetime64[ns] 2014-09-05
Dimensions without coordinates: x, y

Dataset的变量：

In [29]:
ds.data_vars

Data variables:
    temperature    (x, y, time) float64 23.19 16.63 20.29 ... 13.48 0.9469 18.03
    precipitation  (x, y, time) float64 6.163 9.75 3.838 ... 1.989 1.397 4.391

Dataset的坐标：

In [30]:
ds.coords

Coordinates:
    lon             (x, y) float64 -99.83 -99.32 -99.79 -99.23
    lat             (x, y) float64 42.25 42.21 42.63 42.59
  * time            (time) datetime64[ns] 2014-09-06 2014-09-07 2014-09-08
    reference_time  datetime64[ns] 2014-09-05

Dataset的属性：

In [31]:
ds.attrs

{}

In [32]:
ds.attrs['title'] = 'example attribute'
ds

<xarray.Dataset>
Dimensions:         (time: 3, x: 2, y: 2)
Coordinates:
    lon             (x, y) float64 -99.83 -99.32 -99.79 -99.23
    lat             (x, y) float64 42.25 42.21 42.63 42.59
  * time            (time) datetime64[ns] 2014-09-06 2014-09-07 2014-09-08
    reference_time  datetime64[ns] 2014-09-05
Dimensions without coordinates: x, y
Data variables:
    temperature     (x, y, time) float64 23.19 16.63 20.29 ... 0.9469 18.03
    precipitation   (x, y, time) float64 6.163 9.75 3.838 ... 1.989 1.397 4.391
Attributes:
    title:    example attribute

Dataset的长度是指的是变量的个数。

In [37]:
len(ds)

2

如果想要保存 Dataset，最自然的方法自然是保存到 netcdf 文件。

In [39]:
ds.to_netcdf("saved_on_disk.nc")

在xarray中读取netcdf文件可以这样：

In [40]:
ds_disk = xr.open_dataset("saved_on_disk.nc")
ds_disk

<xarray.Dataset>
Dimensions:         (time: 3, x: 2, y: 2)
Coordinates:
  * time            (time) datetime64[ns] 2014-09-06 2014-09-07 2014-09-08
    lon             (x, y) float64 ...
    lat             (x, y) float64 ...
    reference_time  datetime64[ns] ...
Dimensions without coordinates: x, y
Data variables:
    temperature     (x, y, time) float64 ...
    precipitation   (x, y, time) float64 ...
Attributes:
    title:    example attribute